In [13]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import csv
import json
import re
import pickle

from nltk.corpus import stopwords
from pymystem3 import Mystem
import operator


%matplotlib inline

In [14]:
data = pd.read_csv('data/guide_raw.csv')
data.head()

,Unnamed: 0,personnel_num,guide_date,guide_time,conversion,target_3_days,txt
0,0,1512,18-05-2018,14:44,Нет конверсии,0,сейчас скажу своим да и ещё что-то он на сайте...
1,1,1512,21-05-2018,9:46,Конверсия платежа,1,спасибо до свидания хорошо я рада была вам пом...
2,2,1512,25-05-2018,14:25,Конверсия посещения,0,будет готово ставка финансовые есть как это от...
3,3,1820,27-05-2018,12:41,Конверсия платежа,1,сейчас все закрыто хорошо узнаю все делать оче...
4,4,1820,18-05-2018,10:12,Нет конверсии,0,окей справ после а сколько апелляционный понят...


In [15]:
data.loc[0]['txt']

'сейчас скажу своим да и ещё что-то он на сайте карте обязательно зарегистрировались не знаю я вот три брала электроника вот тех на другой а сейчас нажимаете что есть кредиты депозиты карточка депозит потом это пятьдесят тысяч то карточка у вас а вот как информацию по депозитам после открывайте дверь я говорю для того чтобы информацию посмотреть вам надо прийти в отделение а вы можете это чуть дома коммунальные платежи как оплачивать ходит хожу может комиссия вот видите ходите ножками а вы можете тоже дома сидеть оплачивать коммунальные услуги и при этом получать еще бонусы мой на каждый платеж за что бонусами а мой забыли другая ходить это чтобы тебе ножки работает чтобы мы работу эти бонусы потом можно пользоваться также оплачивать давайте посмотрим зарегистрирована или нет номер свой телефон помните пусть семь регистрацию вас три семерки целую пытаетесь честно и мозгами сегодня и семнадцати четыреста сорок девять пятьдесят девять двадцать восемь четыреста сорок девять шестьдесят дев

## Cleaning

In [16]:
m = Mystem()
stop_words = set(stopwords.words('russian'))

badwords = [u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
            u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
            u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
            u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
            u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
            u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок',u'в',
            u'б',u'г',u'д',u'е',u'ж',u'з',u'й',u'к',u'л',u'ф',u'н',u'о',u'п',u'р',u'с',u'т',u'ч',u'ц',u'ч',
            u'ш',u'щ',u'ь',u'ъ',u'ы',u'э','ю']

popluarCountries = [u'казахстан', u'россия',u'узбекистан',u'киргизия',
                    u'сша',u'штаты',u'америка',u'китай',u'туркменистан',
                    u'сирия',u'монголия',u'франция',u'англия',u'турция']

KZCities = [u'алма-ата',u'алматы',u'караганда',u'караганды',
            u'уральск',u'орал',u'усть-каменогорск',u'оскемен',
            u'кокшетау',u'кокчетав',u'семей',u'семипалатинск',
            u'тараз',u'шымкент',u'астана',u'павлодар',
            u'актобе',u'атырау',u'актау',u'кызылорда',
            u'петропавловск',u'талдыкорган',u'костанай',u'шу',
            u'жезказган',u'байконур',u'туркестан',u'экибастуз',
            u'астан',u'столица',u'город',u'столичный',u'рк',
            u'жамбылский',u'алматинский',u'акмолинский',
            u'западный',u'восточный',u'северный',u'южный',
            u'центральный']

Time = [u'январь',u'февраль',u'март',u'апрель',
        u'май',u'июнь',u'июль',u'август',
        u'сентябрь',u'октябрь',u'ноябрь',u'декабрь',
        u'понедельник',u'вторник',u'среда',u'четверг',
        u'пятница',u'суббота',u'воскресенье',u'день',
        'завтра','сегодня','вчера']

Numerals = [u'ноль',u'один',u'два',u'три',
            u'четыре'u'пять',u'шесть',u'семь',
            u'восемь',u'девять',u'десять',u'сто',
            u'тысяча',u'миллион',u'миллиард',u'триллион']

badwords.extend(popluarCountries)
badwords.extend(KZCities)
badwords.extend(Time)
badwords.extend(Numerals)

for word in badwords:
    stop_words.add(word)

### Make function to lemmatizing

In [19]:
def Lemmatisator(data):
    lemma = m.lemmatize(data)
    return ''.join(lemma)

def stopwordsEleminator(data, negation=True):
    for word in stop_words:
        if(word == u'не' or word == u'ни') and (negation == True):
            continue
        if word in data:
            for i in range(data.count(word)):
                data.remove(word)
    return data

def dataCleaner(data, negationConcat = False): # False if clear all, True if clear and concatinate negations
    cleanedData = re.sub("[^а-яА-ЯЁё]"," ", data) # leave only russian text
    cleanedData = Lemmatisator(cleanedData)
    cleanedData = cleanedData.lower().split()
    cleanedData = stopwordsEleminator(cleanedData,negationConcat)
    
    if negationConcat == True:
        for i in range (0, len(cleanedData)-1):
            if(cleanedData[i] == u'не' or cleanedData[i] == u'ни'):
                cleanedData[i+1]=('не'+cleanedData[i+1])
        cleanedData = stopwordsEleminator(cleanedData,False)

    return cleanedData

def duplicateEleminator(data):
    uniqueData = list()
    cnt = 0
    
    for item in data:
        if item not in uniqueData:
            uniqueData.append(item)
        else:
            cnt+=1
    print("Duplicates:{}".format(cnt))        
    return uniqueData        


In [22]:
print(data.loc[0]['txt'])

сейчас скажу своим да и ещё что-то он на сайте карте обязательно зарегистрировались не знаю я вот три брала электроника вот тех на другой а сейчас нажимаете что есть кредиты депозиты карточка депозит потом это пятьдесят тысяч то карточка у вас а вот как информацию по депозитам после открывайте дверь я говорю для того чтобы информацию посмотреть вам надо прийти в отделение а вы можете это чуть дома коммунальные платежи как оплачивать ходит хожу может комиссия вот видите ходите ножками а вы можете тоже дома сидеть оплачивать коммунальные услуги и при этом получать еще бонусы мой на каждый платеж за что бонусами а мой забыли другая ходить это чтобы тебе ножки работает чтобы мы работу эти бонусы потом можно пользоваться также оплачивать давайте посмотрим зарегистрирована или нет номер свой телефон помните пусть семь регистрацию вас три семерки целую пытаетесь честно и мозгами сегодня и семнадцати четыреста сорок девять пятьдесят девять двадцать восемь четыреста сорок девять шестьдесят девя

In [39]:
print(type(dataCleaner(data.loc[0]['txt'])))
print(dataCleaner(data.loc[0]['txt']))

<class 'list'>
['сказать', 'свой', 'сайт', 'карта', 'обязательно', 'зарегистрироваться', 'знать', 'брать', 'электроника', 'нажимать', 'кредит', 'депозит', 'карточка', 'депозит', 'пятьдесят', 'карточка', 'информация', 'депозит', 'открывать', 'дверь', 'говорить', 'информация', 'посмотреть', 'приходить', 'отделение', 'мочь', 'дома', 'коммунальный', 'платеж', 'оплачивать', 'ходить', 'ходить', 'мочь', 'комиссия', 'видеть', 'ходить', 'ножка', 'мочь', 'дома', 'сидеть', 'оплачивать', 'коммунальный', 'услуга', 'получать', 'бонус', 'каждый', 'платеж', 'бонус', 'забывать', 'ходить', 'ножка', 'работать', 'работа', 'бонус', 'пользоваться', 'также', 'оплачивать', 'давать', 'посмотреть', 'зарегистрировать', 'номер', 'свой', 'телефон', 'помнить', 'пусть', 'регистрация', 'семерка', 'целый', 'пытаться', 'честно', 'мозги', 'семнадцать', 'четыреста', 'сорок', 'пятьдесят', 'двадцать', 'четыреста', 'сорок', 'шестьдесят', 'двадцать', 'восемьдесят', 'семерка', 'четыреста', 'сорок', 'шестьдесят', 'двадцать', '

In [26]:
data.head()

,Unnamed: 0,personnel_num,guide_date,guide_time,conversion,target_3_days,txt
0,0,1512,18-05-2018,14:44,Нет конверсии,0,сейчас скажу своим да и ещё что-то он на сайте...
1,1,1512,21-05-2018,9:46,Конверсия платежа,1,спасибо до свидания хорошо я рада была вам пом...
2,2,1512,25-05-2018,14:25,Конверсия посещения,0,будет готово ставка финансовые есть как это от...
3,3,1820,27-05-2018,12:41,Конверсия платежа,1,сейчас все закрыто хорошо узнаю все делать оче...
4,4,1820,18-05-2018,10:12,Нет конверсии,0,окей справ после а сколько апелляционный понят...


In [27]:
data.shape

(1796, 7)

In [41]:
data.shape[0]

1796

In [44]:
data['clean'] = data['txt'].apply(lambda x: dataCleaner(x))

In [45]:
data.head()

,Unnamed: 0,personnel_num,guide_date,guide_time,conversion,target_3_days,txt,clean
0,0,1512,18-05-2018,14:44,Нет конверсии,0,сейчас скажу своим да и ещё что-то он на сайте...,"[сказать, свой, сайт, карта, обязательно, заре..."
1,1,1512,21-05-2018,9:46,Конверсия платежа,1,спасибо до свидания хорошо я рада была вам пом...,"[спасибо, свидание, рад, помогать, весь, добры..."
2,2,1512,25-05-2018,14:25,Конверсия посещения,0,будет готово ставка финансовые есть как это от...,"[готовый, ставка, финансовый, ставка, семнадца..."
3,3,1820,27-05-2018,12:41,Конверсия платежа,1,сейчас все закрыто хорошо узнаю все делать оче...,"[закрывать, узнавать, весь, делать, удобно, ср..."
4,4,1820,18-05-2018,10:12,Нет конверсии,0,окей справ после а сколько апелляционный понят...,"[окей, справа, сколько, апелляционный, понятно..."


In [46]:
data.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
personnel_num,1512,1512,1512,1820,1820
guide_date,18-05-2018,21-05-2018,25-05-2018,27-05-2018,18-05-2018
guide_time,14:44,9:46,14:25,12:41,10:12
conversion,Нет конверсии,Конверсия платежа,Конверсия посещения,Конверсия платежа,Нет конверсии
target_3_days,0,1,0,1,0
txt,сейчас скажу своим да и ещё что-то он на сайте...,спасибо до свидания хорошо я рада была вам пом...,будет готово ставка финансовые есть как это от...,сейчас все закрыто хорошо узнаю все делать оче...,окей справ после а сколько апелляционный понят...
clean,"[сказать, свой, сайт, карта, обязательно, заре...","[спасибо, свидание, рад, помогать, весь, добры...","[готовый, ставка, финансовый, ставка, семнадца...","[закрывать, узнавать, весь, делать, удобно, ср...","[окей, справа, сколько, апелляционный, понятно..."


In [126]:
len(text)

1796

## Vectorization

In [47]:
from gensim.models import word2vec

In [110]:
text = []
sentiment = data['target_3_days']
for i in range(data.shape[0]):
    text.append(data.loc[i]['clean'])
    

num_features = 120
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3


# word2vec
model = word2vec.Word2Vec(text,workers=num_workers, size=num_features, \
                          min_count = min_word_count,window=context, sample=downsampling)
model.init_sims(replace = True)
model_name = "word2vec"
model.save(model_name)

In [111]:
model

In [112]:
model.wv['свой']

array([-0.01087673,  0.03524679,  0.03569096, -0.05058137, -0.07455079,
        0.08096801, -0.05069195, -0.08297034, -0.10869984,  0.09976668,
       -0.00475469, -0.09740144,  0.07230405,  0.11147366,  0.00601158,
        0.11519227, -0.08705989, -0.03561475,  0.0401538 ,  0.12501198,
       -0.15693657, -0.07024452,  0.0144052 ,  0.19586219,  0.1376543 ,
        0.01898795,  0.029666  , -0.0131273 , -0.2823702 , -0.04267832,
        0.04662421, -0.13887444, -0.0720148 ,  0.0075699 , -0.08539051,
        0.17168152,  0.01464036,  0.04313608,  0.07920177,  0.03968018,
       -0.18183057, -0.03416197, -0.12153488,  0.02507146,  0.01212022,
        0.01482901, -0.06908873,  0.06564751,  0.03919216, -0.12221301,
        0.03245661,  0.05993472,  0.00362342,  0.0463728 , -0.1542129 ,
       -0.0625159 , -0.06070189, -0.0357924 , -0.02852016,  0.00051396,
       -0.19353518, -0.06235116, -0.01020414,  0.06327235,  0.1384385 ,
        0.04391693,  0.10612176,  0.08291993,  0.08139142,  0.02

In [113]:
def makeFeatureVec(words,model,num_features):
    featureVec = np.zeros((num_features,), dtype='float32')
    n = 0
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            n+=1
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,n)
    return featureVec

def getAverageVec(posts, model,num_features):
    cnt = 0
    reviewFeatureVecs = np.zeros((len(posts), num_features), dtype='float32')
    
    for sentence in posts:
        reviewFeatureVecs[cnt] = makeFeatureVec(sentence,model,num_features)
        cnt+=1
    return reviewFeatureVecs

In [114]:
DataVecs = getAverageVec(text,model,num_features)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [115]:
np.isnan(DataVecs).any()
np.count_nonzero(np.isnan(DataVecs))
DataVecs = np.nan_to_num(DataVecs)
np.isnan(DataVecs).any()

False

In [116]:
with open("text.pkl", "wb") as f:
    pickle.dump(DataVecs, f)

In [117]:
with open("sentiment.pkl", "wb") as f:
    pickle.dump(sentiment,f)

In [125]:
DataVecs[0]

array([[ 1.0342035e-02,  2.8329287e-02,  6.0120528e-03, ...,
        -1.7609898e-02,  2.7126172e-03, -2.3668019e-02],
       [ 4.9390402e-03,  6.3096364e-03, -3.9581880e-03, ...,
        -1.6565878e-02, -4.0366398e-03, -1.8787084e-02],
       [ 1.9956592e-03,  5.3245593e-03, -2.4505092e-02, ...,
        -1.4046309e-04,  1.5255475e-02, -2.0561153e-02],
       ...,
       [-6.9252667e-03,  5.9566735e-03, -1.8647706e-02, ...,
        -2.0571899e-02, -1.8724995e-02, -1.1533846e-02],
       [ 1.1395164e-02,  1.8316554e-02, -3.6505880e-03, ...,
        -6.1119399e-03, -2.3600098e-03, -2.2698147e-02],
       [ 1.5079380e-02,  4.1704785e-02,  7.0762602e-05, ...,
        -7.8994501e-03, -3.4611113e-03, -2.9684171e-02]], dtype=float32)

In [119]:
model.wv.most_similar('телефон')

[('сотовый', 0.8059059381484985),
 ('мобильный', 0.7303787469863892),
 ('восьмерка', 0.668705940246582),
 ('билайн', 0.6539534330368042),
 ('телефончик', 0.6516900062561035),
 ('логин', 0.6348855495452881),
 ('останавливать', 0.6319443583488464),
 ('предложение', 0.5845174193382263),
 ('скачивать', 0.583716630935669),
 ('начинать', 0.57284015417099)]

## Make model

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
import operator
from sklearn.model_selection import train_test_split
import math
from sklearn import metrics
from gensim.models import word2vec
from sklearn.model_selection import cross_validate

from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [123]:
X_W2V = pickle.load(open("text.pkl","rb"))
y_W2V = pickle.load(open("sentiment.pkl","rb"))

In [124]:
X_W2V_Train, X_W2V_Test, y_W2V_Train, y_W2V_Test = train_test_split(X_W2V,y_W2V, test_size = 0.2, random_state = 42)

In [127]:
Classifier = RandomForestClassifier(n_estimators = 100, max_depth = 5)
Classifier.fit(X_W2V_Train,y_W2V_Train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [128]:
predicted = Classifier.predict(X_W2V_Test)
print("Accuracy", Classifier.score(X_W2V_Test,y_W2V_Test))
print("Metrics", metrics.classification_report(predicted,y_W2V_Test))

Accuracy 0.825
Metrics              precision    recall  f1-score   support

          0       1.00      0.82      0.90       360
          1       0.00      0.00      0.00         0

avg / total       1.00      0.82      0.90       360



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
